In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install ollama

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

Unsloth 2024.10.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Preparation

In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def format_prompt(sample):
    return {
        "prompt": alpaca_prompt.format(sample["instruction"], sample["input"], ""),
        "response": sample["output"]
    }

from datasets import load_dataset
dataset = load_dataset("tatsu-lab/alpaca")["train"]
dataset = dataset.map(format_prompt)

<a name="Train"></a>
### Training the Model

In [5]:
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="prompt",
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        output_dir="unsloth_model",
        report_to="none",
    ),
    packing=True,
)

trainer.train()

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,829 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


Step,Training Loss
1,1.296600
2,1.331400
3,1.261500
4,1.222500
5,1.239400
6,1.200100
7,1.188000
8,1.131100
9,1.070500
10,1.061900


TrainOutput(global_step=60, training_loss=0.7962285896142324, metrics={'train_runtime': 183.9571, 'train_samples_per_second': 2.609, 'train_steps_per_second': 0.326, 'total_flos': 2.225661850681344e+16, 'train_loss': 0.7962285896142324, 'epoch': 0.1696113074204947})

<a name="Inference"></a>
### Inference

In [6]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Describe the process of photosynthesis",
        "Include the role of chlorophyll and the main inputs and outputs",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the process of photosynthesis

### Input:
Include the role of chlorophyll and the main inputs and outputs

### Response:
<|end_of_text|>


<a name="Export"></a>
### Exporting to Ollama

In [7]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Installing ollama to /usr/local
100 13320    0 13320    0     0  33206      0 --:--:-- --:--:-- --:--:-- 33216
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [8]:
model.save_pretrained_gguf("unsloth_model", tokenizer, quantization_method="q8_0")

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 62.48 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 60.11it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at unsloth_model into q8_0 GGUF format.
The output location will be /content/unsloth_model/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: unsloth_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> Q8_0, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_n

In [9]:
# import subprocess
# import os

# subprocess.Popen(["ollama", "serve"])

# # Check if the GGUF file exists
# gguf_file = "unsloth_model/model-unsloth-Q8_0.gguf"
# if not os.path.exists(gguf_file):
#     print(f"GGUF file not found at {gguf_file}")
#     print("Contents of unsloth_model directory:")
#     print(os.listdir("unsloth_model"))
# else:
#     print(f"GGUF file found at {gguf_file}")

In [10]:
# # Create a basic Modelfile for Ollama
# modelfile_content = f"""
# FROM {gguf_file}
# PARAMETER stop "<|endoftext|>"
# PARAMETER stop "<|im_end|>"
# PARAMETER stop "<|im_start|>"
# """

# with open("./unsloth_model/Modelfile", "w") as f:
#     f.write(modelfile_content)

# print("Modelfile created successfully.")
# print("Modelfile contents:")
# print(modelfile_content)

In [11]:
# !ollama create unsloth_model -f ./unsloth_model/Modelfile

In [12]:
# from ollama import Client
# client = Client()

# response = client.chat(model='unsloth_model', messages=[{
#     'role': 'user',
#     'content': 'Explain the concept of artificial intelligence'
# }])

# print(response['message']['content'])

In [13]:
# import subprocess
# import os
# import time

# # Start Ollama server in the background
# subprocess.Popen(["ollama", "serve"], shell=True)

# # Correct GGUF file name
# gguf_file = os.path.abspath("./unsloth_model/unsloth.Q8_0.gguf")  # Use correct GGUF file name
# if not os.path.exists(gguf_file):
#     print(f"GGUF file not found at {gguf_file}")
#     print("Contents of unsloth_model directory:")
#     print(os.listdir("unsloth_model"))
# else:
#     print(f"GGUF file found at {gguf_file}")

# # Create a basic Modelfile for Ollama with correct stop parameters
# modelfile_content = f"""
# FROM {gguf_file} # Use absolute path in Modelfile
# PARAMETER stop "<|endoftext|>"
# PARAMETER stop "<|im_end|>"
# PARAMETER stop "<|im_start|>"
# """

# with open("./unsloth_model/Modelfile", "w") as f:
#     f.write(modelfile_content)

# print("Modelfile created successfully.")
# print("Modelfile contents:")
# print(modelfile_content)

# # Create the model in Ollama
# !ollama create unsloth_model -f ./unsloth_model/Modelfile

# # Wait for a longer period for the model and server to load
# time.sleep(15)

# from ollama import Client
# client = Client()

# try:
#     response = client.chat(model='unsloth_model', messages=[{
#         'role': 'user',
#         'content': 'Explain the concept of artificial intelligence'
#     }])

#     print(response['message']['content'])

# except Exception as e:
#     print(f"An error occurred: {e}")
#     print("Please ensure the Ollama server is running and the model is loaded correctly.")

In [15]:
!zip -r /content/unsloth_model.zip /content/unsloth_model/


  adding: content/unsloth_model/ (stored 0%)
  adding: content/unsloth_model/model-00004-of-00004.safetensors (deflated 21%)
  adding: content/unsloth_model/special_tokens_map.json (deflated 71%)
  adding: content/unsloth_model/generation_config.json (deflated 34%)
  adding: content/unsloth_model/tokenizer.json (deflated 74%)
  adding: content/unsloth_model/model-00003-of-00004.safetensors (deflated 23%)
  adding: content/unsloth_model/tokenizer_config.json (deflated 96%)
  adding: content/unsloth_model/Modelfile (deflated 30%)
  adding: content/unsloth_model/model-00001-of-00004.safetensors (deflated 23%)
  adding: content/unsloth_model/config.json (deflated 48%)
  adding: content/unsloth_model/model-00002-of-00004.safetensors (deflated 23%)
  adding: content/unsloth_model/checkpoint-60/ (stored 0%)
  adding: content/unsloth_model/checkpoint-60/special_tokens_map.json (deflated 71%)
  adding: content/unsloth_model/checkpoint-60/tokenizer.json (deflated 74%)
  adding: content/unsloth_m

In [18]:
from google.colab import files
files.download("/content/unsloth_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from google.colab import files
model_path = "/content/unsloth_model/unsloth.Q8_0.gguf"
files.download(model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Interactive ChatGPT-style Interface

To interact with the model in a ChatGPT-style interface, open a new terminal and run:

```
ollama run unsloth_model
```

You can then have a conversation with the model. Press Ctrl+D to exit.